# TAOR project

## Setup

In the cell below we will load the core libraries we will be using.

In [1]:
# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# Pyomo
from pyomo.environ import *

# Misc libraries
import os

# Scripting library
import get_basura as gb

# Optimization models

The models are instantiated as Pyomo objects and are solved using the NEOS server. 

## Linear programming model

This model is solved using either CPLEX or MOSEK model

In [2]:
# Max distances
demax = 25
dlmax = 125

# Bring the data as dictionaries
q_j, dist_jk_dist_jl, dist_kl, w_jk0, f_jk_f_jl, g_kl = gb.get_data(demax, dlmax)

# Parameters
c_c = 0.045
c_u = 0.128571

q = sum(q_j.values())

# s_k = 204400 # original value from Dr Antunes
s_k = 182500 # 500 ton/day capacity

m = 1E6

# Step 0: Instantiate a model object
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Step 1: Define index sets
J = list(q_j.keys())
K = list(q_j.keys())
L = list(q_j.keys())
J1 = ["Arouca", "Estarreja", "Oliveira de Azemeis", "Sao Joao da Madeira", "Sever do Vouga", "Gois", "Lousa", "Pampilhosa da Serra", "Penela", "Vila Nova Poiares", "Ansiao", "Castanheira de Pera", "Pedrogao Grande"]
K1 = ["Estarreja", "Oliveira de Azemeis", "Sever do Vouga", "Gois", "Pampilhosa da Serra", "Ansiao"]

# Step 2: Define the decision variables
model.w_jk = Var(J, K, within= Binary)
model.v_jl = Var(J, L, within=Binary)
model.y_k = Var(K, within=Binary)
model.z_l = Var(L, within=Binary)
model.x_kl = Var(K,L, domain = NonNegativeReals)

# Step 3: Objective function
def obj_rule(model):
    return sum( c_u * dist_jk_dist_jl[j,k] * q_j[j] * model.w_jk[j,k] for j in J for k in K)+\
        sum( c_u * dist_jk_dist_jl[j,l] * q_j[j] * model.v_jl[j,l] for j in J for l in L)+\
        sum( c_c * dist_kl[k,l] * model.x_kl[k,l] for k in K for l in L) #+ sum( m*model.y_k[k] for k in K)

model.Cost = Objective(rule=obj_rule, sense = minimize)

# Step 4: Constraints              
def rule_1(model,J):
    return sum( model.w_jk[J,k] for k in K ) + \
           sum( model.v_jl[J,l] for l in L ) == 1 
    
def rule_2(model,K):
    return sum( q_j[j]*model.w_jk[j, K] for j in J ) == sum( model.x_kl[K,l] for l in L )  
    
def rule_3(model,J,K):
    return model.w_jk[J,K] <= f_jk_f_jl[J,K]*model.y_k[K]

def rule_4(model,J,L):
    return model.v_jl[J,L] <= f_jk_f_jl[J,L]*model.z_l[L]
   
def rule_5(model,K,L):
    return model.x_kl[K,L] <= g_kl[K,L]*q*model.z_l[L]

def rule_6(model,K):
    return sum(q_j[j]*model.w_jk[j,K] for j in J)<=s_k*model.y_k[K]

def rule_7(model):
    return sum(model.z_l[l] for l in L)==1

def rule_8(model, J1, K1):
    return model.w_jk[J1,K1] == w_jk0[J1, K1]

def rule_9(model): #experiment, incinerator not in Agueda
    return (model.z_l["Agueda"] == 0)

def rule_10(model): #experiment, incinerator not in Mealheada
    return (model.z_l["Mealhada"] == 0)

def rule_11(model): #experiment, incinerator not in Anadia
    return (model.z_l["Anadia"] == 0)

def rule_12(model, J1, K1):
    return model.y_k[K1] >= w_jk0[J1, K1]

def rule_13(model):
    return sum(model.y_k[k] for k in K) <= 9

def rule_14(model, J):
    return sum(model.w_jk[J, k] for k in K) <= 9

model.C_1 = Constraint( J, rule=rule_1 )
model.C_2 = Constraint( K, rule=rule_2 )
model.C_3 = Constraint( J, K, rule=rule_3 )
model.C_4 = Constraint( J, L, rule=rule_4 )
model.C_5 = Constraint( K, L, rule = rule_5)
model.C_6 = Constraint( K, rule = rule_6 )
model.C_7 = Constraint( rule = rule_7)
model.C_8 = Constraint(J1, K1, rule = rule_8) 
#model.C_9 = Constraint( rule = rule_9) # experiment, incinerator not in Agueda
#model.C_10 = Constraint( rule = rule_10) # experiment, incinerator not in Mealhada
#model.C_11 = Constraint( rule = rule_11) # experiment, incinerator not in Anadia
#model.c_12 = Constraint( J1, K1, rule = rule_12) # allow the municipalities to go to other ts
model.c_13 = Constraint( rule = rule_13) # limit number of transfer stations without using the m term
#model.c_14 = Constraint( J, rule = rule_14) # limit number of municipalities assigned to a ts

# Call Mosel and solve
#results = SolverFactory('amplxpress').solve(model)

# Call the NEOS server and use CPLEX for solving

# email address
os.environ['NEOS_EMAIL'] = 's2123659@ed.ac.uk'

solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt="cplex")

results.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4076
  Number of variables: 3960
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CPLEX 20.1.0.0\x3a optimal integer solution; objective 1327415.8206406198; 465 MIP simplex iterations; 0 branch-and-bound nodes; absmipgap = 2.32831e-10, relmipgap = 1.75401e-16
  Termination condition: optimal
  Id: 2
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

### Results

In [3]:
# Output detailed solution
header = "Transfer stations"
print(f"\n{header}")
print(f"="*len(header))

for k in K:
    if(model.y_k[k]() == 1):
        print(k, model.y_k[k]())

for l in L:
    if(model.z_l[l]() == 1):
        print("Incinerator is in: ", l, model.z_l[l]())
        
if 'ok' == str(results.Solver.status):
    t_cost = model.Cost() - m*sum(model.y_k[k]() for k in K)
    print(f"Optimal value = €{model.Cost():.2f}")
    print(f"Transportation costs = €{t_cost:.2f}")
else:
    print("No Valid Solution Found")


Transfer stations
Estarreja 1.0
Ilhavo 1.0
Oliveira de Azemeis 1.0
Sever do Vouga 1.0
Coimbra 1.0
Gois 1.0
Montemor-o-Velho 1.0
Pampilhosa da Serra 1.0
Ansiao 1.0
Incinerator is in:  Agueda 1.0
Optimal value = €1327415.82
Transportation costs = €-7672584.18


## Visualization maps

Here are some auxiliary lists for displaying the maps

In [4]:
# Fill lists with the optimization results 

# Transfer stations
y = []

for k in K:
    if(model.y_k[k]() == 1):
        y.append(k)

#Incinerator
z = []

for l in L:
    if(model.z_l[l]() == 1):
        z.append(l)

# Link between municipalities and transfer stations
links_ts = []

for k in K:
    for j in J:
        if(model.w_jk[j, k]() == 1):
            links_ts.append((k, j))

# Link between municipalities and the incinerator
links_inc = []

for l in L:
    for j in J:
        if(model.v_jl[j, l]() == 1):
            links_inc.append((l, j))

# Get the coordinates
ts_new, ts_exist, inc, mun, w_jk, v_jl = gb.get_coord(y, z, links_ts, links_inc, K1)

### Display the map

In [5]:
# Create map instance and display

centro_map = gb.create_map(ts_new, ts_exist, inc, mun, w_jk, v_jl)
centro_map

# Modified model (Robust optimization)

In [126]:
def get_new_data(demax, dlmax, year):

    currDir = os.getcwd()
    dataAntunes = os.path.join(currDir, 'data_antunes.xls')
    new_data = os.path.join(currDir, "data_antunes_new.xls")

    antunesSheet0 = pd.read_excel(dataAntunes, sheet_name=0, header = 1)
    antunesSheet1 = pd.read_excel(dataAntunes, sheet_name=1, header = 2)
    antunesSheet2 = pd.read_excel(dataAntunes, sheet_name=2, header = 2)
    antunesSheet4 = pd.read_excel(dataAntunes, sheet_name=4, header = None)

    municipalities = antunesSheet0.copy()
    municipalities = municipalities.drop(columns = ["ORD_COD", "Q (ton/dia)", "Q (ton/ano)", "Pop 2001", "ET exist.", "ET assign."])
    
    # New data
    waste_df = pd.read_excel(new_data, sheet_name=0, header = 1)
    q_j_init = waste_df.drop(columns = ["ORD_COD", "Concelho", "Q (ton/dia)", "Q_2001", "Pop 2001", "Q_2015", "Pop_2015", "ET exist.", "ET assign."])

    distmatrix1 = antunesSheet1.copy()
    d_jk_d_jl_init = distmatrix1.drop(columns = ["Unnamed: 0"])
    d_jk_d_jl_init.columns = list(range(0, 36))

    distmatrix2 = antunesSheet2.copy()
    d_kl_init = distmatrix2.drop(columns = ["Unnamed: 0"])

    w_jk0_init = antunesSheet4.copy()

    # Create f_jk_f_jl matrices: 
    # If the distances between municipalities and transfer stations  d_jk <= demax or w_jk =1, then 1. Else, 0.
    f_jk_f_jl_init = d_jk_d_jl_init.copy()
    w_jk0_init.sort_index(inplace=True) == d_jk_d_jl_init.sort_index(inplace = True)

    f_jk_f_jl_init[(f_jk_f_jl_init <= demax) | (w_jk0_init == 1)] = 1
    f_jk_f_jl_init[f_jk_f_jl_init != 1]= 0
    f_jk_f_jl_init = f_jk_f_jl_init.astype(int)

    # Take the transposed matrix instead as wjk0 an f_jk_f_jl are not symmetric 
    w_jk0_init = w_jk0_init.T
    f_jk_f_jl_init = f_jk_f_jl_init.T

    # Create g_kl matrix: 
    # If the distances between transfer stations to incinerator d_kl <= dlmax, then 1. Else, 0.
    g_kl_init = d_kl_init.copy()
   
    g_kl_init = g_kl_init.where(d_kl_init > dlmax, 1)
    g_kl_init[g_kl_init != 1] = 0
    g_kl_init = g_kl_init.astype(int)

    # Create temp dictionaries
    q_j_init_dict = q_j_init.to_dict(orient = "list")
    d_jk_d_jl_init_dict = d_jk_d_jl_init.to_dict(orient = "list")
    d_kl_init_dict = d_kl_init.to_dict(orient = "list")
    w_jk0_init_dict = w_jk0_init.to_dict(orient = "list")
    f_jk_f_jl_init_dict = f_jk_f_jl_init.to_dict(orient = "list")
    g_kl_init_dict = g_kl_init.to_dict(orient = "list")

    # Get the base_keys for the dictionaries
    mun_list = municipalities.squeeze().to_list()
    base_keys = [(i, j) for i in mun_list for j in mun_list]

    # Get the matrix values 
    q_j = [value for key, value in q_j_init_dict.items()]
    q_j_list = [j for i in q_j for j in i]

    d_jk_d_jl = [value for key, value in d_jk_d_jl_init_dict.items()]
    d_jk_d_jl_list = [j for i in d_jk_d_jl for j in i]

    d_kl = [value for key, value in d_kl_init_dict.items()]
    d_kl_list = [j for i in d_kl for j in i]

    w_jk0 = [value for key, value in w_jk0_init_dict.items()]
    w_jk0_list = [j for i in w_jk0 for j in i]

    f_jk_f_jl = [value for key, value in f_jk_f_jl_init_dict.items()]
    f_jk_f_jl_list = [j for i in f_jk_f_jl for j in i]

    g_kl = [value for key, value in g_kl_init_dict.items()]
    g_kl_list = [j for i in g_kl for j in i]

    # create final dictionaries
    q_j_dict = dict(zip(mun_list, q_j_list))
    d_jk_d_jl_dict = dict(zip(base_keys, d_jk_d_jl_list))
    d_kl_dict = dict(zip(base_keys, d_kl_list))
    w_jk0_dict = dict(zip(base_keys, w_jk0_list))
    f_jk_f_jl_dict = dict(zip(base_keys, f_jk_f_jl_list))
    g_kl_dict = dict(zip(base_keys, g_kl_list))

    return q_j_dict, d_jk_d_jl_dict, d_kl_dict, w_jk0_dict, f_jk_f_jl_dict, g_kl_dict

In [123]:
# Bring the data as dictionaries
muns, _, _, _, _, _ = get_data(demax, dlmax)

mun_list = [i for i in muns.keys()]

In [128]:
# Max distances
demax = 25
dlmax = 125

# Bring the data as dictionaries
q_j, dist_jk_dist_jl, dist_kl, w_jk0, f_jk_f_jl, g_kl = get_new_data(demax, dlmax, year = "2015")

# Parameters
c_c = 0.045
c_u = 0.128571

q = sum(q_j.values())

# s_k = 204400 # original value from Dr Antunes
s_k = 182500 # 500 ton/day capacity

m = 1E6

# Step 0: Instantiate a model object
model_rob = ConcreteModel()
model_rob.dual = Suffix(direction=Suffix.IMPORT)

# Step 1: Define index sets
J = list(q_j.keys())
K = list(q_j.keys())
L = list(q_j.keys())
J1 = ["Arouca", "Estarreja", "Oliveira de Azemeis", "Sao Joao da Madeira", "Sever do Vouga", "Gois", "Lousa", "Pampilhosa da Serra", "Penela", "Vila Nova Poiares", "Ansiao", "Castanheira de Pera", "Pedrogao Grande"]
K1 = ["Estarreja", "Oliveira de Azemeis", "Sever do Vouga", "Gois", "Pampilhosa da Serra", "Ansiao"]

# Step 2: Define the decision variables
model_rob.w_jk = Var(J, K, within= Binary)
model_rob.v_jl = Var(J, L, within=Binary)
model_rob.y_k = Var(K, within=Binary)
model_rob.z_l = Var(L, within=Binary)
model_rob.x_kl = Var(K,L, domain = NonNegativeReals)

# Step 3: Objective function
def obj_rule(model_rob):
    return sum( c_u * dist_jk_dist_jl[j,k] * q_j[j] * model_rob.w_jk[j,k] for j in J for k in K)+\
        sum( c_u * dist_jk_dist_jl[j,l] * q_j[j] * model_rob.v_jl[j,l] for j in J for l in L)+\
        sum( c_c * dist_kl[k,l] * model_rob.x_kl[k,l] for k in K for l in L) #+ sum( m*model.y_k[k] for k in K)

model_rob.Cost = Objective(rule=obj_rule, sense = minimize)

# Step 4: Constraints              
def rule_1(model_rob,J):
    return sum( model_rob.w_jk[J,k] for k in K ) + \
           sum( model_rob.v_jl[J,l] for l in L ) == 1 
    
def rule_2(model_rob,K):
    return sum( q_j[j]*model_rob.w_jk[j, K] for j in J ) == sum( model_rob.x_kl[K,l] for l in L )  
    
def rule_3(model_rob,J,K):
    return model_rob.w_jk[J,K] <= f_jk_f_jl[J,K]*model_rob.y_k[K]

def rule_4(model_rob,J,L):
    return model_rob.v_jl[J,L] <= f_jk_f_jl[J,L]*model_rob.z_l[L]
   
def rule_5(model_rob,K,L):
    return model_rob.x_kl[K,L] <= g_kl[K,L]*q*model_rob.z_l[L]

def rule_6(model_rob,K):
    return sum(q_j[j]*model_rob.w_jk[j,K] for j in J)<=s_k*model_rob.y_k[K]

def rule_7(model_rob):
    return sum(model_rob.z_l[l] for l in L)==1

def rule_8(model_rob, J1, K1):
    return model_rob.w_jk[J1,K1] == w_jk0[J1, K1]

def rule_9(model_rob): #experiment, incinerator not in Agueda
    return (model.z_l["Agueda"] == 0)

def rule_10(model_rob): #experiment, incinerator not in Mealheada
    return (model_rob.z_l["Mealhada"] == 0)

def rule_11(model_rob): #experiment, incinerator not in Anadia
    return (model_rob.z_l["Anadia"] == 0)

def rule_12(model_rob, J1, K1):
    return model_rob.y_k[K1] >= w_jk0[J1, K1]

def rule_13(model_rob):
    return sum(model_rob.y_k[k] for k in K) <= 9

def rule_14(model_rob, J):
    return sum(model_rob.w_jk[J, k] for k in K) <= 9

model_rob.C_1 = Constraint( J, rule=rule_1 )
model_rob.C_2 = Constraint( K, rule=rule_2 )
model_rob.C_3 = Constraint( J, K, rule=rule_3 )
model_rob.C_4 = Constraint( J, L, rule=rule_4 )
model_rob.C_5 = Constraint( K, L, rule = rule_5)
model_rob.C_6 = Constraint( K, rule = rule_6 )
model_rob.C_7 = Constraint( rule = rule_7)
model_rob.C_8 = Constraint(J1, K1, rule = rule_8) 
#model_rob.C_9 = Constraint( rule = rule_9) # experiment, incinerator not in Agueda
#model_rob.C_10 = Constraint( rule = rule_10) # experiment, incinerator not in Mealhada
#model_rob.C_11 = Constraint( rule = rule_11) # experiment, incinerator not in Anadia
#model_rob.c_12 = Constraint( J1, K1, rule = rule_12) # allow the municipalities to go to other ts
model_rob.c_13 = Constraint( rule = rule_13) # limit number of transfer stations without using the m term
#model_rob.c_14 = Constraint( J, rule = rule_14) # limit number of municipalities assigned to a ts

# Call Mosel and solve
#results = SolverFactory('amplxpress').solve(model_rob)

# Call the NEOS server and use MOSEK for solving

# email address
os.environ['NEOS_EMAIL'] = 's2123659@ed.ac.uk'

solver_manager = SolverManagerFactory('neos')
results_rob = solver_manager.solve(model_rob, opt="cplex")

results_rob.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4076
  Number of variables: 3960
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CPLEX 20.1.0.0\x3a optimal integer solution; objective 1054622.0474321428; 462 MIP simplex iterations; 0 branch-and-bound nodes
  Termination condition: optimal
  Id: 2
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solu

In [129]:
# Output detailed solution
header = "Transfer stations"
print(f"\n{header}")
print(f"="*len(header))

for k in K:
    if(model_rob.y_k[k]() == 1):
        print(k, model_rob.y_k[k]())

for l in L:
    if(model_rob.z_l[l]() == 1):
        print("Incinerator is in: ", l, model_rob.z_l[l]())
        
if 'ok' == str(results_rob.Solver.status):
    t_cost = model_rob.Cost() - m*sum(model_rob.y_k[k]() for k in K)
    print(f"Optimal value = €{model_rob.Cost():.2f}")
    print(f"Transportation costs = €{t_cost:.2f}")
else:
    print("No Valid Solution Found")


Transfer stations
Estarreja 1.0
Ilhavo 1.0
Oliveira de Azemeis 1.0
Sever do Vouga 1.0
Coimbra 1.0
Gois 1.0
Montemor-o-Velho 1.0
Pampilhosa da Serra 1.0
Ansiao 1.0
Incinerator is in:  Mealhada 1.0
Optimal value = €1054622.05
Transportation costs = €-7945377.95


In [124]:
for j in J:
    for k in K:
        print(model_rob.w_jk[j, k]())
print(mun_list)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-9.922035666235132e-16
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.649307715003294e-16
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

## Map

In [130]:
# Fill lists with the optimization results 

# Transfer stations
y = []

for k in K:
    if(model_rob.y_k[k]() == 1):
        y.append(k)

#Incinerator
z = []

for l in L:
    if(model_rob.z_l[l]() == 1):
        z.append(l)

# Link between municipalities and transfer stations
links_ts = []

for k in K:
    for j in J:
        if(model_rob.w_jk[j, k]() == 1):
            links_ts.append((k, j))

# Link between municipalities and the incinerator
links_inc = []

for l in L:
    for j in J:
        if(model_rob.v_jl[j, l]() == 1):
            links_inc.append((l, j))

# Get the coordinates
ts_new, ts_exist, inc, mun, w_jk, v_jl = get_coord(y, z, links_ts, links_inc, K1)

In [131]:
print(w_jk)

                     ts     lat_ts   long_ts                  mun    lat_mun  \
30               Ansiao  39.916667 -8.433333      Pedrogao Grande  39.916667   
28               Ansiao  39.916667 -8.433333  Castanheira de Pera  40.000000   
27               Ansiao  39.916667 -8.433333               Ansiao  39.916667   
26               Ansiao  39.916667 -8.433333           Alvaiazere  39.833333   
25               Ansiao  39.916667 -8.433333               Penela  40.033333   
29               Ansiao  39.916667 -8.433333  Figueiro dos Vinhos  39.900000   
16              Coimbra  40.250000 -8.450000             Penacova  40.266667   
14              Coimbra  40.250000 -8.450000      Condeixa-a-Nova  40.116667   
13              Coimbra  40.250000 -8.450000              Coimbra  40.250000   
15              Coimbra  40.250000 -8.450000     Miranda do Corvo  40.100000   
0             Estarreja  40.750000 -8.566667            Estarreja  40.750000   
1             Estarreja  40.750000 -8.56

In [132]:
print(v_jl)

        inc    lat_inc  long_inc                 mun    lat_mun  long_mun
0  Mealhada  40.383333     -8.45              Agueda  40.574444 -8.448056
1  Mealhada  40.383333     -8.45              Anadia  40.433333 -8.433333
2  Mealhada  40.383333     -8.45            Mealhada  40.383333 -8.450000
3  Mealhada  40.383333     -8.45  Oliveira do Bairro  40.516667 -8.500000
4  Mealhada  40.383333     -8.45          Cantanhede  40.346240 -8.594070


### Display the map


In [133]:
# Create a map of the Centro region of Portugal
centroLat = 40.784142221076074
centroLong = -8.12884084353569

# Instantiate feature groups for the transfer stations, the incinerator and the municipalities
ts_group = folium.map.FeatureGroup(name = "New transfer stations")
ts_e_group = folium.map.FeatureGroup(name = "Existing transfer stations")
inc_group = folium.map.FeatureGroup(name = "Incinerator")
mun_group = folium.map.FeatureGroup(name = "Municipalities")

# Add each ts, municipalities and the incinerator to the feature groups
for lat, lng, in zip(ts_new.lat, ts_new.long):
    ts_group.add_child(
        folium.CircleMarker([lat, lng], radius=5, color='green', fill=True, fill_color='green', fill_opacity=0.7))
for lat, lng, in zip(ts_exist.lat, ts_exist.long):
    ts_e_group.add_child(
        folium.CircleMarker([lat, lng], radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.7))
for lat, lng, in zip(mun.lat, mun.long):
    mun_group.add_child(
        folium.CircleMarker([lat, lng], radius=2, color='black', fill=True, fill_color='black', fill_opacity=0.7))
for lat, lng, in zip(inc.lat, inc.long):
    inc_group.add_child(
        folium.CircleMarker([lat, lng], radius=6, color='red', fill=True, fill_color='red', fill_opacity=0.7))

# Add the feature groups to the map
centroMap = folium.Map(location=[centroLat, centroLong], zoom_start=8)
centroMap.add_child(ts_group)
centroMap.add_child(ts_e_group)
centroMap.add_child(mun_group)
centroMap.add_child(inc_group)

# Add a layer control to the map and other tile layer options
folium.TileLayer('cartodbpositron').add_to(centroMap)
folium.TileLayer('stamentoner').add_to(centroMap)
folium.map.LayerControl('topright', collapsed=False).add_to(centroMap)

# Add the graph lines for the links
w_jk_link = list(zip(zip(w_jk.lat_ts, w_jk.long_ts), zip(w_jk.lat_mun, w_jk.long_mun)))
v_jl_link = list(zip(zip(v_jl.lat_inc, v_jl.long_inc), zip(v_jl.lat_mun, v_jl.long_mun)))

folium.PolyLine(w_jk_link, color="darkred", weight=1.5, opacity=1).add_to(centroMap)
folium.PolyLine(v_jl_link, color="darkred", weight=1.5, opacity=1).add_to(centroMap)

# Display the map
centroMap